## Tratamento da regionalização da execução orçamentária

Nesse notebook são tratados os dados da regionalização da execução orçamentária da Prefeitura Municipal de São Paulo para o ano de 2024

In [ ]:
import pandas as pd
import numpy as np
from utils.load_csv import load_csv
from utils.load_shapefile import load_shapefile
from utils.save_csv import save_csv

In [ ]:
df_reg_orcamento = load_csv("reg_orcamento_2024.csv")

#### Filtro regionalizável

Filtra a base de dados para conter apenas despesas regionalizáveis

In [ ]:
df_reg_orcamento["TIPO_REGIONALIZAÇÃO"].unique()

In [ ]:
df_orcamento_regionalizavel = df_reg_orcamento[df_reg_orcamento["TIPO_REGIONALIZAÇÃO"] == "Despesa Regionalizável"].copy()

df_orcamento_regionalizavel["TIPO_REGIONALIZAÇÃO"].unique()

In [ ]:
df_orcamento_regionalizavel

#### Transformação da variável de gasto

Transforma a variável de gasto da base de dados de string para float

In [ ]:
df_orcamento_regionalizavel["VALOR_DETALHAMENTO_AÇÃO"].unique()

In [ ]:
df_orcamento_regionalizavel["VALOR_DETALHAMENTO_AÇÃO"] = df_orcamento_regionalizavel["VALOR_DETALHAMENTO_AÇÃO"].str.replace(".", "").str.replace(",", ".").astype(float)

pd.options.display.float_format = '{:,.2f}'.format

df_orcamento_regionalizavel["VALOR_DETALHAMENTO_AÇÃO"].describe()

In [ ]:
np.set_printoptions(suppress = True)

df_orcamento_regionalizavel["VALOR_DETALHAMENTO_AÇÃO"].unique()

#### Normalização das subprefeituras

Trata os nomes das subprefreituras na base de dados da regionalização da execução orçamentária para estar igual aos nomes no shapefile das subprefeituras

In [ ]:
df_orcamento_regionalizavel["SUBPREFEITURA"].unique()

In [ ]:
supra_subprefs = [
    "Supra Subprefeitura Centro",
    "Supra Subprefeitura Leste",
    "Supra Subprefeitura Norte",
    "Supra Subprefeitura Oeste",
    "Supra Subprefeitura Sul"]

df_orcamento_regionalizavel = df_orcamento_regionalizavel[~df_orcamento_regionalizavel["SUBPREFEITURA"].isin(supra_subprefs)]

df_orcamento_regionalizavel["SUBPREFEITURA"].unique()

In [ ]:
gdf = load_shapefile("subprefeituras_geosampa")

In [ ]:
gdf.head()

In [ ]:
gdf["sp_nome"].unique()

In [ ]:
mapper = {"Subprefeitura Sé": "SE",
          "Subprefeitura Vila Mariana": "VILA MARIANA",
          "Subprefeitura Santana/Tucuruvi": "SANTANA-TUCURUVI",
          "Subprefeitura Campo Limpo": "CAMPO LIMPO",
          "Subprefeitura Itaim Paulista": "ITAIM PAULISTA",
          "Subprefeitura Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
          "Subprefeitura Sapopemba": "SAPOPEMBA",
          "Subprefeitura Capela do Socorro": "CAPELA DO SOCORRO",
          "Subprefeitura Lapa": "LAPA",
          "Subprefeitura Jaçanã/Tremembé": "JACANA-TREMEMBE",
          "Subprefeitura Penha": "PENHA",
          "Subprefeitura São Miguel Paulista": "SAO MIGUEL",
          "Subprefeitura Perus/Anhanguera": "PERUS",
          "Subprefeitura Butantã": "BUTANTA",
          "Subprefeitura de Guaianases": "GUAIANASES",
          "Subprefeitura São Mateus": "SAO MATEUS",
          "Subprefeitura Cidade Ademar": "CIDADE ADEMAR",
          "Subprefeitura de Vila Prudente": "VILA PRUDENTE",
          "Subprefeitura Ipiranga": "IPIRANGA",
          "Subprefeitura Pinheiros": "PINHEIROS",
          "Subprefeitura Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
          "Subprefeitura Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
          "Subprefeitura M'Boi Mirim": "M BOI MIRIM",
          "Subprefeitura Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
          "Subprefeitura Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
          "Subprefeitura Santo Amaro": "SANTO AMARO",
          "Subprefeitura Mooca": "MOOCA",
          "Subprefeitura Ermelino Matarazzo": "ERMELINO MATARAZZO",
          "Subprefeitura Cidade Tiradentes": "CIDADE TIRADENTES",
          "Subprefeitura Itaquera": "ITAQUERA",
          "Subprefeitura Jabaquara": "JABAQUARA",
          "Subprefeitura Parelheiros": "PARELHEIROS"}

df_orcamento_regionalizavel.loc[:, "SUBPREFEITURA"] = df_orcamento_regionalizavel["SUBPREFEITURA"].map(mapper)

df_orcamento_regionalizavel["SUBPREFEITURA"].unique()

In [ ]:
save_csv(df_orcamento_regionalizavel, "orcamento_tratado_2024.csv")